In [1]:
import  requests, os, sys, json, urllib, shutil, re, pickle
from textwrap import fill
from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

class color:
    MAGENTA     = '\033[35m'
    CYAN        = '\033[36m'
    BLUE        = '\033[34m'
    BRIGHT      = '\033[37m'
    DARKYELLOW  = '\033[33m'
    GREEN       = '\033[32m'
    DARKRED     = '\033[31m'
    DULL        = '\033[30m'
    PURPLE      = '\033[95m'
    DARKCYAN    = '\033[36m'
    RED         = '\033[91m'
    BOLD        = '\033[1m'
    ITALICS     = '\033[3m'
    UNDERLINE   = '\033[4m'
    END         = '\033[0m'
    def getMagenta(string):
        return color.MAGENTA + string + color.END
        
    def getCyan(string):
        return color.CYAN + string + color.END
        
    def getBlue(string):
        return color.BLUE + string + color.END
        
    def getBright(string):
        return color.BRIGHT + string + color.END
        
    def getDarkYellow(string):
        return color.DARKYELLOW + string + color.END
        
    def getGreen(string):
        return color.GREEN + string + color.END
        
    def getRed(string):
        return color.RED + string + color.END
        
    def getItalics(string):
        return color.ITALICS + string + color.END
        
    def getUnderline(string):
        return color.UNDERLINE + string + color.END
        
    def getBold(string):
        return color.BOLD + string + color.END
        
    def getDarkRed(string):
        return color.DARKRED + string + color.END
        
    def getDull(string):
        return color.Dull + string + color.END
        
    def getPurple(string):
        return color.PURPLE + string + color.END
        
    def getDarkCyan(string):
        return color.DARKCYAN + string + color.END

def alignBullets(left, right):
    screen_width = shutil.get_terminal_size().columns - 15
    wrapped = fill(right, width=screen_width, subsequent_indent=' '*12)
    return '  {0:<10}{1}'.format(left, wrapped)

def wrapLineNumberDfn(right):
    screen_width = shutil.get_terminal_size().columns - 15
    wrapped = fill(right, width=screen_width, subsequent_indent=' '*11)
    return wrapped

def printAligned(left, right):
    screen_width = shutil.get_terminal_size().columns - 20
    wrapped = fill(right, width=screen_width, subsequent_indent=' '*15)
    return '  {0:>13}{1}'.format(left, wrapped)

def printSynAntons(left, right):
    screen_width = shutil.get_terminal_size().columns - 25
    wrapped = fill(right, width=screen_width, subsequent_indent=' '*21)
    return '  {0:>13}{1}'.format(left, wrapped)

In [2]:
chromedriver = "/Users/chaser/Projects/mybin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [96]:
word = 'odyssey'#'offhand'#'bode' #'apparition' 
url = 'https://www.google.co.in/search?q=define%20' + word + '&expnd=1'

driver.get(url)
html = driver.page_source

In [97]:
soup = BeautifulSoup(html, "lxml")
new = soup.select("div.lr_dct_ent")

In [46]:
new[0]

<div class="lr_dct_ent vmod" data-hveid="42"><div class="vk_ans" style="margin-bottom:0;line-height:normal"><span data-dobid="hdw">offhand</span></div><div class="vmod" style="max-height:none"><div class="vmod" style="max-height:none"><div class="lr_dct_sf_h"><i><span>adjective</span></i></div><div class="xpdxpnd vk_gy" data-mh="-1" style="max-height:none"><span>adjective: <b>offhand</b></span><span>; adjective: <b>off-hand</b></span></div><div><span class="lr_dct_ph"><span>ɒfˈhand</span>,<span>ˈɒfhand</span>/</span><span class="lr_dct_spkr lr_dct_spkr_off" data-ved="0ahUKEwjVma3VhtnaAhVLro8KHbxrAWUQlfQBCCswAA" jsaction="dob.p" title="Listen"><input height="14" src="" type="image" width="14"/><audio data-dobid="aud" onloadedmetadata="this.parentNode.style.display = 'inline-block'" preload="auto" src="//ssl.gstatic.com/dictionary/static/sounds/20160317/offhand--_gb_1.mp3"></audio></span></div><ol class="lr_dct_sf_sens"><li><div class="vmod" style="max-height:none"><div class="lr_dct_sf_

In [47]:
aqq = None
for block in new:
        name = block.find("span", {"data-dobid":"hdw"})
        subBlock_POS = block.find_all("div", {"class":"lr_dct_sf_h"})
        subBlock_pronun = block.find_all("span", {"class":"lr_dct_ph"})  # less than or equal to the no of sub blocks(POS)
        subBlock_lineBelowPOS = block.find_all("div", {"class":"xpdxpnd vk_gy"})
        subBlock_crux = block.find_all("ol", {"class":"lr_dct_sf_sens"})
#         print(subBlock_crux)
        subBlock_wordForm = block.find("div", {"class":"vmod vk_gy"})   # e.g. in words like 'edgewise' which are directly not present in google dictionary "adverb: edgewise" was missing so for that line.
        print(color.getMagenta(name.text.title()), end = "") # print the word for the block
        # print the pronunciation with the word in the block if there is only one otherwise print it with the sub-block content for that use a flag
        pronun_done_flag = False
        if len(subBlock_pronun) == 1:
            pronun_done_flag = True
            print(color.getCyan(" /" + subBlock_pronun[0].text + ","), end="")
        # print(len(subBlock_POS))
        iterator = 0
        for subName in subBlock_POS:
            if iterator == 0:
                print(" " + color.getRed(color.getItalics(subName.getText(separator=u' '))), end="")   # the prints the part of speech which defines a sub block
            else:
                print("    " + color.getRed(color.getItalics(subName.getText(separator=u' '))), end="")   # the prints the part of speech which defines a sub block        
            if iterator < len(subBlock_lineBelowPOS):
                print(" | " + subBlock_lineBelowPOS[iterator].text, end="")
            if (pronun_done_flag == False) and (iterator < len(subBlock_pronun)):
                print("    /" + subBlock_pronun[iterator].text, end="")
            if subBlock_wordForm != None:
                print("\n\t" + subBlock_wordForm.text, end="")
            # print("\t" + subBlock_crux.text)
            lines = subBlock_crux[iterator].find_all("div", {"class":"lr_dct_sf_sen vk_txt"})
#             print("==>",lines)
            iterator = iterator + 1
            # print(iterator)
            for line in lines: # line means the line that has a number assigned to it some lines are a part of the line assigned the number in a bulleted list class="_Jig"
#                 print(line, "\n")
                number = line.find("strong")
                bullets = line.find_all("div", {"class":"PNlCoe"})
                another = 0 # becasue the first dfn will be printed next to the number
                if number != None:
                    print("\n      " + color.getGreen(number.text + ".") + " ", end="")
                else:
                    print()
                    another = 1 # since number is not there so no need of this flag
                for bullet in bullets:
#                     print(bullet, "\n")
                    dfn = bullet.find("div", {"data-dobid":"dfn"}) # the definition in the bullet
                    first_eg = bullet.find("div", {"class":"vk_gy"}) # the first example of the bullet
                    syns_antons = bullet.find_all("table", {"class":"vk_tbl vk_gy"})
                    if another == 0: # also the printing of the first bullet will be different
                        print(color.getBright(wrapLineNumberDfn(dfn.text)))
                        another = 1
                    else:
                        print(alignBullets("        \u2022", color.getBright(dfn.text)))
                        # print("\t  " + "\u2022 ", end="")
                        # print(color.getBright(dfn.text))
                    if first_eg != None: #Sometimes the example just after the definition is absent
                        print("\t   " + first_eg.text)
                    if syns_antons != None:
                        if aqq is None:
                            aqq = syns_antons
                        for item in syns_antons:
                            title = item.find("td", {"class":"lr_dct_nyms_ttl"})
                            nyms = item.find_all("span")
                            eg = item.find("div", {"class":"vk_gy"})
                            # print("\t   " + color.getItalics(title.text) + " " ,end="")
                            syn_antons_title = "\t   " + color.getItalics(title.text) + " "
                            syn_antons_words = set()
                            for nym in nyms:
                                if nym.text[-1] != "\"" and nym.text != 'More': # since taking all the span will get the examples also and since examples end in " therefore we need to skip them
                                    syn_antons_words.add(nym.text.strip('\n').strip('\t').strip('  ').strip(','))# + nym.text.strip('\n').strip('\t').strip('  ') + ' '  # print(nym.text, end="")
                            sinn_words = ''
                            for t in syn_antons_words:
                                if t is not "":
                                    sinn_words = sinn_words + t + ', '
                            print(printSynAntons(syn_antons_title, sinn_words), end="")
                            if eg != None:
                                print("\n\t\t    " + eg.text)
                            else:
                                print()


Offhand adjective | adjective: offhand; adjective: off-hand    /ɒfˈhand,ˈɒfhand/
      1. ungraciously or offensively nonchalant or cool in manner.
	   "you were a bit offhand with her this afternoon"
  	   synonyms: ;, unceremonious, take-it-or-leave-it, spontaneous, casual, nonchalant, distant, off-the-cuff, cool, perfunctory, indifferent, terse, unstudied, couldn't
                     -care-less, extempore, unthinking, cavalier, offhanded, impromptu, aloof, careless, impolite, pococurante, unpremeditated, blasé,
                     uninterested, brusque, off, offish, dismissive, rude, unconcerned, ungracious, insouciant, extemporaneous, cursory, curt, abrupt,
                     discourteous, uncivil, glib,
		    "an offhand comment that she regretted almost immediately"
    adverb | adverb: offhand; adverb: off-hand    /ɒfˈhand/
      1. without previous thought or consideration.
	   "I can't think of a better answer offhand"
  	   synonyms: ;, spontaneously, just like that, on 

In [48]:
ttttt
for item in aqq:
    title = item.find("td", {"class":"lr_dct_nyms_ttl"})
    nyms = item.find_all("span")
    eg = item.find("div", {"class":"vk_gy"})
#     print("\t   " + color.getItalics(title.text) + " " ,end="")
    syn_antons_title = "\t   " + color.getItalics(title.text) + " "
    syn_antons_words = set()
#     print(eg)
    for nym in nyms:
        if nym.text[-1] != "\"" and nym.text != 'More': # since taking all the span will get the examples also and since examples end in " therefore we need to skip them
            syn_antons_words.add(nym.text.strip('\n').strip('\t').strip('  ').strip(',').strip(';'))# + nym.text.strip('\n').strip('\t').strip('  ') + ' '  # print(nym.text, end="")
    sinn_words = ''
    for t in syn_antons_words:
        if t is not "":
            sinn_words = sinn_words + t + ', '
    print(printSynAntons(syn_antons_title, sinn_words), end="")
#     print(syn_antons_words)
    ttttt = syn_antons_words
    if eg != None:
        print("\n\t\t    " + eg.text)
    else:
        print()

NameError: name 'ttttt' is not defined

In [49]:
# ttttt = 'extemporaneously,'
new_list_syn = set()
for t in ttttt:
#     print(t.strip(','))
    new_list_syn.add(t.strip(','))
    
# print(new_list_syn)

sinn = ''
for t in new_list_syn:
    if t is not '':
        sinn = sinn + t + ', '
print(sinn)

NameError: name 'ttttt' is not defined

In [50]:
a = new[0].find_all("div", {})

In [51]:
a[-1].text

'"I can\'t think of a better answer offhand"'

In [52]:
b = new[0].find("div", {"style","max-height:none"})

In [53]:
b

In [54]:
RESULTS_LOCATOR = "//span[@class='lr_dct_trns_sel_cnt']"
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, RESULTS_LOCATOR)))
options = driver.find_element(By.XPATH, RESULTS_LOCATOR)
opt = options.find_element(By.XPATH, '//Select')
for o in opt.find_elements_by_tag_name('option'):
    if o.text == 'Hindi':
        o.click() # select() in earlier versions of webdriver
        break

In [55]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [56]:
trans = soup.find("div", {"class":"lr_dct_trns vmod"})

In [57]:
trans

<div class="lr_dct_trns vmod" data-hveid="89" style="padding-bottom:10px"><span class="lr_dct_trns_h vk_sh vk_gy">Translate offhand to<span class="lr_dct_trns_sel_cnt"><select class="lr_dct_trns_sel lr_dct_trns_sel_browser vk_bk vk_txt ElpxJd" data-log-string="translations-language-select" data-pref-link="/setprefs?sig=0_TZLtvEsOP9PU-CvD6XNZVzgnrm0%3D&amp;prev=https://www.google.co.in/search?q%3Ddefine%2Boffhand%26expnd%3D1%26dtrnscllps%3D1%26brd%3D1524783899586000" data-ved="0ahUKEwjVma3VhtnaAhVLro8KHbxrAWUQmp0CCFowAA" id="tl_select" jsaction="dob.l;change:dob.ctl"><option value="">Choose language</option><option value="af">Afrikaans</option><option value="sq">Albanian</option><option value="am">Amharic</option><option value="ar">Arabic</option><option value="hy">Armenian</option><option value="az">Azerbaijani</option><option value="bn">Bangla</option><option value="eu">Basque</option><option value="be">Belarusian</option><option value="bs">Bosnian</option><option value="bg">Bulgarian

In [58]:
translations = trans

In [59]:
posHeading = translations.find_all("div", {"class":"lr_dct_tg_pos vk_txt"})
transWords = translations.find_all("ol", {"class":"lr_dct_tg_trns"})

In [60]:
print(len(posHeading))
print(len(transWords))

0
0


In [61]:
posHeading

[]

In [62]:
transWords

[]

In [63]:
translation_table = []
translation_heading_subBlock = []
# I don't understand why but sometimes these come out to be empty.
for pos in posHeading:
    translation_heading_subBlock.append(pos.text)
# print("also")
translation_word_subBlock = {}
subBlockMinLength = 100
listNumber = 1
maxLen = 0
for trans in transWords:
    wordList = trans.get_text('||').split('||')
    wordList = [x for x in wordList if not (x.isdigit() or x[0] == '.')]
    # print(wordList)
    if len(wordList) < subBlockMinLength:
        subBlockMinLength = len(trans)
    wordNumber = 1
    for listWord in wordList:
        # print (listWord)
        if len(listWord) > maxLen:
            maxLen = len(listWord)
        translation_word_subBlock[listNumber, wordNumber] = listWord
        wordNumber = wordNumber + 1
    listNumber = listNumber + 1

# print("atleast")
translation_table = []
translation_table.append(translation_heading_subBlock)
for nWord in range(1, subBlockMinLength+1):
    tempList = []
    for nList in range(1, listNumber):
        tempList.append(translation_word_subBlock.get((nList, nWord)))
    # print(tempList)
    translation_table.append(tempList)

offset = maxLen + 10

print(translation_heading_subBlock, translation_word_subBlock)
# I don't understand why but sometimes only blank spaces are printed.
prevent_just_blank = False
for t in translation_word_subBlock:
    if translation_word_subBlock.get(t) != ' ':
        prevent_just_blank = True
        break
if prevent_just_blank:
    for i in range(len(translation_table)):
        for j in range(len(translation_table[0])):
            if translation_table[i][j] == None:
                translation_table[i][j] = ""
            if i == 0:
                translation_table[i][j] = color.getUnderline(color.getBlue(translation_table[i][j]))
            indented = ' {:{align}{width}}'.format(color.getDarkRed(translation_table[i][j]), align='^', width=offset)
            print(" " +  indented, end="")
        print()

[] {}


### Merriam webster

In [64]:
import  requests, urllib, shutil
# from textwrap import fill
# from collections import Counter
from bs4 import BeautifulSoup

In [65]:
word = 'run'#'amenable'
url = 'https://www.merriam-webster.com/dictionary/' + word

In [66]:
response = requests.get(url, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:49.0) Gecko/20100101 Firefox/49.0"})
html = response.content
vocab_soup = BeautifulSoup(html, 'html.parser')

In [67]:
texts = vocab_soup.findAll(text=True)

def visible(element):
    if element.parent.name in ['style', 'body', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)):
        return False
    return True

visible_texts = filter(visible, texts)

In [68]:
vocab_soup.find("div", {"class":"card-box full-def-box secondary-card def-text "}).findAll("li")

AttributeError: 'NoneType' object has no attribute 'findAll'

In [ ]:
print(visible_texts)

In [ ]:
defi = vocab_soup.find("ol", {"class":"definition-list"})

In [ ]:
defi

In [ ]:
print(defi.text)

In [ ]:
ll = defi.find_all("li")

In [ ]:
ll

In [ ]:
import re
for l in ll:
    span = l.find_all("span")
    for s in span:
        if len(s) > 2:
            eg = s.find("em", {"class":"vi"})
            if eg != None:
                defa = s.text
                ega = eg.text
                defa = re.sub(ega, '', defa)
                print (defa, end="")
                print ("-" ,eg.text)            

Merriam Webster

In [69]:
import urllib.request
from bs4 import BeautifulSoup

In [70]:
html_learners = html_dictionary = ''
html_response = ''
key_dictionary = '20025eb3-dba1-4a47-a254-73f284b7abc5'
key_learners = '4b99aba1-21c0-4a98-9150-7b114c4751b9'

In [71]:
xml_dict = 'http://www.dictionaryapi.com/api/v1/references/collegiate/xml/amenable?key=' + key_dictionary
xml_learners = ' http://www.dictionaryapi.com/api/v1/references/learners/xml/run?key=' + key_learners

In [72]:
with urllib.request.urlopen(xml_dict) as response:
    html_response = response
    html_dictionary = response.read()
with urllib.request.urlopen(xml_learners) as response:
    html_learners = response.read()

In [73]:
html_dictionary.title()

b'<?Xml Version="1.0" Encoding="Utf-8" ?>\r\n<Entry_List Version="1.0">\n\t<Entry Id="Amenable"><Ew>Amenable</Ew><Hw>Ame*Na*Ble</Hw><Sound><Wav>Amenab02.Wav</Wav></Sound><Pr>\xc9\x99-\xcb\x88M\xc4\x93-N\xc9\x99-B\xc9\x99L, -\xcb\x88Me-</Pr><Fl>Adjective</Fl><Et>Anglo-French, From <It>Amener</It> To Bring, Compel, From <It>A-</It> (From Latin <It>Ad-</It>) + <It>Mener</It> To Lead, From Late Latin <It>Minare</It> To Drive, From Latin <It>Minari</It> To Threaten <Ma>Mount</Ma></Et><Def><Date>1596</Date> <Sn>1</Sn> <Dt>:<D_Link>Liable</D_Link> To Be Brought To Account :<Sx>Answerable</Sx> <Vi>Citizens <It>Amenable</It> To The Law</Vi></Dt> <Sn>2 A</Sn> <Dt>:Capable Of Submission (As To Judgment Or Test) :<Sx>Suited</Sx> <Vi>The Data Is <It>Amenable</It> To Analysis</Vi></Dt>  <Sn>B</Sn> <Dt>:Readily Brought To <D_Link>Yield</D_Link>, Submit, Or Cooperate <Vi>A Government Not <It>Amenable</It> To Change</Vi></Dt>  <Sn>C</Sn> <Dt>:<Sx>Willing <Sxn>1</Sxn></Sx> <Vi>Was <It>Amenable</It> To S

In [74]:
# soup_dict = BeautifulSoup(html_dictionary.decode('utf-8', 'ignore'), "lxml")
soup_dict = BeautifulSoup(html_dictionary, "xml")
soup_learn = BeautifulSoup(html_learners, "xml")

In [75]:
print( soup_dict.prettify())

<?xml version="1.0" encoding="utf-8"?>
<entry_list version="1.0">
 <entry id="amenable">
  <ew>
   amenable
  </ew>
  <hw>
   ame*na*ble
  </hw>
  <sound>
   <wav>
    amenab02.wav
   </wav>
  </sound>
  <pr>
   ə-ˈmē-nə-bəl, -ˈme-
  </pr>
  <fl>
   adjective
  </fl>
  <et>
   Anglo-French, from
   <it>
    amener
   </it>
   to bring, compel, from
   <it>
    a-
   </it>
   (from Latin
   <it>
    ad-
   </it>
   ) +
   <it>
    mener
   </it>
   to lead, from Late Latin
   <it>
    minare
   </it>
   to drive, from Latin
   <it>
    minari
   </it>
   to threaten
   <ma>
    mount
   </ma>
  </et>
  <def>
   <date>
    1596
   </date>
   <sn>
    1
   </sn>
   <dt>
    :
    <d_link>
     liable
    </d_link>
    to be brought to account :
    <sx>
     answerable
    </sx>
    <vi>
     citizens
     <it>
      amenable
     </it>
     to the law
    </vi>
   </dt>
   <sn>
    2 a
   </sn>
   <dt>
    :capable of submission (as to judgment or test) :
    <sx>
     suited
    </sx>
 

In [76]:
print(soup_learn.prettify())

<?xml version="1.0" encoding="utf-8"?>
<entry_list version="1.0">
 <entry id="run[1]">
  <hw highlight="yes" hindex="1">
   run
  </hw>
  <sound>
   <wav>
    run00001.wav
   </wav>
  </sound>
  <pr>
   ˈrʌn
  </pr>
  <fl>
   verb
  </fl>
  <in>
   <if>
    runs
   </if>
  </in>
  <in>
   <if>
    ran
   </if>
   <sound>
    <wav>
     run00002.wav
    </wav>
   </sound>
   <pr>
    ˈræn
   </pr>
  </in>
  <in>
   <if>
    run
   </if>
  </in>
  <in>
   <if>
    run*ning
   </if>
  </in>
  <def>
   <sn>
    1
   </sn>
   <sgram>
    no obj
   </sgram>
   <sn>
    a
   </sn>
   <dt>
    :to move with your legs at a speed that is faster than walking
    <vi>
     How fast can you
     <it>
      run
     </it>
     ?
    </vi>
    <vi>
     He
     <it>
      runs
     </it>
     faster than anyone else on the team.
    </vi>
    <vi>
     She
     <it>
      ran
     </it>
     up the stairs to get her jacket.
    </vi>
    <vi>
     We
     <it>
      ran
     </it>
     for the train—

In [77]:
import xml.dom.minidom

xml = xml.dom.minidom.parse(urllib.request.urlopen(xml_dict)) # or xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = xml.toprettyxml()

In [78]:
ss = BeautifulSoup(xml.toprettyxml(), "xml")

In [79]:
print(ss)

<?xml version="1.0" encoding="utf-8"?>
<entry_list version="1.0">
<entry id="amenable">
<ew>amenable</ew>
<hw>ame*na*ble</hw>
<sound>
<wav>amenab02.wav</wav>
</sound>
<pr>ə-ˈmē-nə-bəl, -ˈme-</pr>
<fl>adjective</fl>
<et>
			Anglo-French, from 
			<it>amener</it>
			 to bring, compel, from 
			<it>a-</it>
			 (from Latin 
			<it>ad-</it>
			) + 
			<it>mener</it>
			 to lead, from Late Latin 
			<it>minare</it>
			 to drive, from Latin 
			<it>minari</it>
			 to threaten 
			<ma>mount</ma>
</et>
<def>
<date>1596</date>
<sn>1</sn>
<dt>
				:
				<d_link>liable</d_link>
				 to be brought to account :
				<sx>answerable</sx>
<vi>
					citizens 
					<it>amenable</it>
					 to the law
				</vi>
</dt>
<sn>2 a</sn>
<dt>
				:capable of submission (as to judgment or test) :
				<sx>suited</sx>
<vi>
					the data is 
					<it>amenable</it>
					 to analysis
				</vi>
</dt>
<sn>b</sn>
<dt>
				:readily brought to 
				<d_link>yield</d_link>
				, submit, or cooperate 
				<vi>
					a government 

In [80]:
soup_dict.find("et").text

'Anglo-French, from amener to bring, compel, from a- (from Latin ad-) + mener to lead, from Late Latin minare to drive, from Latin minari to threaten mount'

In [81]:
print(soup_dict.find("et").text)

Anglo-French, from amener to bring, compel, from a- (from Latin ad-) + mener to lead, from Late Latin minare to drive, from Latin minari to threaten mount


In [82]:
print(pretty_xml_as_string)

<?xml version="1.0" ?>
<entry_list version="1.0">
	
	
	<entry id="amenable">
		<ew>amenable</ew>
		<hw>ame*na*ble</hw>
		<sound>
			<wav>amenab02.wav</wav>
		</sound>
		<pr>ə-ˈmē-nə-bəl, -ˈme-</pr>
		<fl>adjective</fl>
		<et>
			Anglo-French, from 
			<it>amener</it>
			 to bring, compel, from 
			<it>a-</it>
			 (from Latin 
			<it>ad-</it>
			) + 
			<it>mener</it>
			 to lead, from Late Latin 
			<it>minare</it>
			 to drive, from Latin 
			<it>minari</it>
			 to threaten 
			<ma>mount</ma>
		</et>
		<def>
			<date>1596</date>
			 
			<sn>1</sn>
			 
			<dt>
				:
				<d_link>liable</d_link>
				 to be brought to account :
				<sx>answerable</sx>
				 
				<vi>
					citizens 
					<it>amenable</it>
					 to the law
				</vi>
			</dt>
			 
			<sn>2 a</sn>
			 
			<dt>
				:capable of submission (as to judgment or test) :
				<sx>suited</sx>
				 
				<vi>
					the data is 
					<it>amenable</it>
					 to analysis
				</vi>
			</dt>
			  
			<sn>b</sn>
			 
			<dt>
				:readily brough

In [83]:
entries = soup_dict.find_all("entry")

In [84]:
isinstance(entries, bs4.element.NavigableString)

NameError: name 'bs4' is not defined

In [85]:
def print_if_element_not_none(ele): # options like same line, next line, with a tab
    if ele is not None:
        print(ele.text)

In [86]:
def process_one_sn(next_tag, pre_type): #prev_type to validate type is a<class 'bs4.element.Tag'> and not a navigableString
    while next_tag.name != "sn":
        if next_tag.name == "sd":
            print("; " + next_tag.text + " ", end="")
        else:
            print(next_tag.text, end="")
        if next_tag is None:
            return None
        next_tag = next_tag.next_sibling#.next_sibling
        while pre_type != type(next_tag):
            try:
                next_tag = next_tag.next_sibling
            except:
                return None
#         while next_tag == "":
#             next_tag = next_tag.next_sibling.next_sibling
#             print("ajay =====------=======---1", next_tag)
        if pre_type != type(next_tag):
            next_tag = next_tag.previous_sibling
#             print("ajay =====------=======---1",next_tag)
            return next_tag
        if next_tag is None:
            return None
#         elif "vt" in next_tag.previous_sibling.name:
#             return next_tag.previous_sibling
#         print("ajay =====------=======---2",next_tag)
#         print(type(next_tag.previous_sibling))
        if next_tag is not None and "dro" in next_tag.name:
            break
        
    print()
    print(next_tag.text, " ", end="")
    next_tag = next_tag.next_sibling.next_sibling
    return next_tag

In [87]:
def do_verbs(vt_form):
    vt_iter = 0
    vt_iter_tag = ''
    for vt in vt_form:
            date = def_item.find("date")
            if vt_form is not None:
                print("   ", vt.text, end="")
            if date is not None:
                print("  ", date.text)
            else:
                print()
            sn_start = def_item.find("sn")
            if vt_iter is 0:
                next_tag = sn_start
            else:
                next_tag = vt_iter_tag
            if sn_start is not None:
                print(sn_start.text, " ", end="")
#                 print(type(sn_start))
                next_tag = next_tag.next_sibling.next_sibling
#                 print(next_tag.name)
                while next_tag is not None and "dro" not in next_tag.name:
                    next_tag = process_one_sn(next_tag, type(next_tag))
                    if next_tag is None or "vt" in next_tag.name:
                        break
#                 while next_tag.name
#                 while next_tag.name != "sn":
#                     if next_tag.name == "sd":
#                         print("; " + next_tag.text + " ", end="")
#                     else:
#                         print(next_tag.text, end="")
#                     next_tag = next_tag.next_sibling.next_sibling
            print()
            vt_iter += 1
            vt_iter_tag = next_tag
            if next_tag is None:
#                 print("(----------------------------------------------)")
                break

spawn; also :  such

In [88]:
def rest_one_sn(next_tag, pre_type):
    while next_tag.name != "sn":
#         if next_tag.name == "sn":
#             print(next_tag.text, end="")
        if next_tag.name == "sd":
            print("; " + next_tag.text, end="")
        else:
            print(" " + next_tag.text)#, end="")
        next_tag = next_tag.next_sibling#.next_sibling
        while pre_type != type(next_tag):
            try:
                next_tag = next_tag.next_sibling
            except:
#                 print("+++++++++++++++gottya------")
                return None
#     print("1st loop broke---", next_tag)
    print(next_tag.text, " ", end="")
#     next_tag = next_tag.next_sibling.next_sibling
    return next_tag

In [89]:
def do_rest(next_tag, pre_type):
    if next_tag is not None and next_tag.name == "sn":
        print(next_tag.text, end="")
    while next_tag is not None:
        next_tag = next_tag.next_sibling.next_sibling
        next_tag = rest_one_sn(next_tag, pre_type)
#         elif next_tag.name == "sd":
#             print("; " + next_tag.text, end="")
#         else:
#             print(" " + next_tag.text)#, end="")
#         next_tag = next_tag.next_sibling#.next_sibling
#         while pre_type != type(next_tag):
#             try:
#                 next_tag = next_tag.next_sibling
#             except:
#                 print("+++++++++++++++gottya------")
#                 return None
        
#     return None

In [90]:
for entry in entries:
    hw_entry_name = entry.find("hw")
    fl_part_of_speech = entry.find("fl")
    if hw_entry_name is not None:
        print("\t",hw_entry_name.text, end="")
    if fl_part_of_speech is not None:
        print(" \t", fl_part_of_speech.text, end="")
        if "verb" in fl_part_of_speech:
            forms = entry.find_all("in")
            if forms is not None:
                print(",", end="")
                for f in forms:
                    print(" ", f.find("if").text + ";", end="")
            print()
        else:
            print()
    else:
        print()
    def_block = entry.find_all("def")
#     print("-", len(def_block), "-") 
    for def_item in def_block:
#         vt_iter = 0
#         vt_iter_tag = ''
        if "verb" in fl_part_of_speech.text:
            vt_form = def_item.find_all("vt")
            do_verbs(vt_form)
        else:
            date = def_item.find("date")
            next_tag = date
            pre_type = type(date)
            if date is not None:
#                 next_tag = date.next_sibling.next_sibling
                next_tag = date.next_sibling#.next_sibling
                while pre_type != type(next_tag):
                    next_tag = next_tag.next_sibling
                print("  ", date.text)#, type(next_tag), next_tag, next_tag.name)
            if next_tag is not None and next_tag.name == "sn":
                do_rest(next_tag, type(next_tag))
            elif next_tag is not None: #just one meaning
                print(next_tag.text)
        print()
            # break when the next siblling is vt
        # noun adj etc will not have a vt so handle that case also
#         print(def_item.text, " - ")
    
et_origin = soup_dict.find("et")
if et_origin is not None:
    print(et_origin.text)
    

	 ame*na*ble 	 adjective
   1596
1 :liable to be brought to account :answerable citizens amenable to the law
2 a   :capable of submission (as to judgment or test) :suited the data is amenable to analysis
b   :readily brought to yield, submit, or cooperate a government not amenable to change
c   :willing 1 was amenable to spending more time at home
 responsible, obedient

Anglo-French, from amener to bring, compel, from a- (from Latin ad-) + mener to lead, from Late Latin minare to drive, from Latin minari to threaten mount


In [91]:
aaa = type(9)

In [92]:
bbb= type(3)

In [93]:
aaa == bbb

True

For words like Odyssey

In [104]:
odyssey = soup.find("span", {"class":"Y0NH2b CLPzrc"})

In [105]:
odyssey

<span class="Y0NH2b CLPzrc"><b>odyssey</b>. Ever since Homer's epic poem The <b>Odyssey</b> told the story of warrior <b>Odysseus</b>' ten-year journey home from Troy, <b>odyssey</b> has meant any epic journey. As with the word journey, <b>odyssey</b> has both a literal meaning and a figurative one.</span>

In [103]:
".".join(odyssey.text.split(".")[1:])

" Ever since Homer's epic poem The Odyssey told the story of warrior Odysseus' ten-year journey home from Troy, odyssey has meant any epic journey. As with the word journey, odyssey has both a literal meaning and a figurative one."

In [102]:
color.getMagenta(odyssey.text.split(".")[0])

'\x1bodyssey\x1b'